In [1]:
import pandas as pd

In [2]:
# amy's list
files = ["../../zq-sample-data/zeek-ndjson/" + i 
         for i in [ "conn.ndjson",
                    "http.ndjson",
                    "files.ndjson",
                    "dns.ndjson",
                    "ssl.ndjson",
                    "weird.ndjson",
                    "syslog.ndjson",
                    "rdp.ndjson",
                    "ntp.ndjson",
                    "smtp.ndjson"]
        ]

In [3]:
dfs = [pd.read_json(f, lines=True) for f in files]

### Analytics queries

I. Count total number of records with each distinct source IP

In [4]:
_field = "id.orig_h"
pd.concat([f[[_field]] for f in dfs 
           if _field in f.columns]) \
  .groupby(_field)\
  .size()\
  .reset_index(name='count')

,id.orig_h,count
0,10.0.0.111,4044
1,10.0.0.205,38
2,10.0.0.208,81
3,10.0.0.219,126
4,10.0.0.227,54439
...,...,...
659,172.30.255.2,19
660,172.31.255.3,200
661,172.31.255.5,2507
662,204.42.254.5,1


### Search queries

I. Find all records with IP 10.128.0.19, sort by timestamp, and return the first 5

In [5]:
# records in a single dataframe
_dfs = pd.concat([f for f in dfs 
           if _field in f.columns], sort=False)
_dfs[_dfs["id.orig_h"] == "10.128.0.19"].sort_values("ts").head(5)

,_path,_write_ts,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,duration,...,xmt_time,num_exts,helo,last_reply,path,tls,fuids,is_webmail,mailfrom,rcptto
19940,conn,2018-03-24T17:15:46.403161Z,2018-03-24T17:15:22.405489Z,C4XMvY3AbHWJ1ejqyd,10.128.0.19,58563.0,10.47.27.186,80.0,tcp,18.997580,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,http,2018-03-24T17:15:22.433287Z,2018-03-24T17:15:22.409738Z,C4XMvY3AbHWJ1ejqyd,10.128.0.19,58563.0,10.47.27.186,80.0,NaN,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182092,conn,2018-03-24T17:18:18.377152Z,2018-03-24T17:17:22.470658Z,CxU6Zp2LgS9Dn0Zw2,10.128.0.19,58604.0,10.47.27.186,80.0,tcp,50.906348,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13900,http,2018-03-24T17:17:22.502126Z,2018-03-24T17:17:22.474844Z,CxU6Zp2LgS9Dn0Zw2,10.128.0.19,58604.0,10.47.27.186,80.0,NaN,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
494741,conn,2018-03-24T17:24:29.273572Z,2018-03-24T17:19:23.463206Z,CEX5va1wKylFQqaKC7,10.128.0.19,58641.0,10.47.27.186,80.0,tcp,5.810212,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# records as dataframes
def search(field, label, key, n):
    rows, results = list(), list()
    for f in dfs:
        if field in f.columns:
            rows += f[f[field] == label].iterrows()

    return sorted(rows, key=lambda x: x[1][key])[:n]

records = search("id.orig_h", "10.128.0.19", "ts", 5)
for r in records:
    display(r[1].to_frame().transpose())

,_path,_write_ts,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,service
19940,conn,2018-03-24T17:15:46.403161Z,2018-03-24T17:15:22.405489Z,C4XMvY3AbHWJ1ejqyd,10.128.0.19,58563,10.47.27.186,80,tcp,18.9976,1303,544,SF,0,ShADTadtfF,12,3254,10,1624,http


,_path,_write_ts,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,trans_depth,method,...,resp_mime_types,referrer,orig_fuids,orig_mime_types,origin,proxied,resp_filenames,username,info_code,info_msg
105,http,2018-03-24T17:15:22.433287Z,2018-03-24T17:15:22.409738Z,C4XMvY3AbHWJ1ejqyd,10.128.0.19,58563,10.47.27.186,80,1,POST,...,[text/json],http://10.47.27.186/wordpress/wp-admin/theme-e...,[FJWmvt4dKqo4Q0iFxd],[text/plain],http://10.47.27.186,NaN,NaN,NaN,NaN,NaN


,_path,_write_ts,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,service
182092,conn,2018-03-24T17:18:18.377152Z,2018-03-24T17:17:22.470658Z,CxU6Zp2LgS9Dn0Zw2,10.128.0.19,58604,10.47.27.186,80,tcp,50.9063,1303,544,SF,0,ShADTadtfF,14,3334,12,1728,http


,_path,_write_ts,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,trans_depth,method,...,resp_mime_types,referrer,orig_fuids,orig_mime_types,origin,proxied,resp_filenames,username,info_code,info_msg
13900,http,2018-03-24T17:17:22.502126Z,2018-03-24T17:17:22.474844Z,CxU6Zp2LgS9Dn0Zw2,10.128.0.19,58604,10.47.27.186,80,1,POST,...,[text/json],http://10.47.27.186/wordpress/wp-admin/theme-e...,[F9FLRlKCsMHV92CM3],[text/plain],http://10.47.27.186,NaN,NaN,NaN,NaN,NaN


,_path,_write_ts,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,service
494741,conn,2018-03-24T17:24:29.273572Z,2018-03-24T17:19:23.463206Z,CEX5va1wKylFQqaKC7,10.128.0.19,58641,10.47.27.186,80,tcp,5.81021,1303,544,S3,0,ShADTadtf,14,3358,12,1728,http


### Data discovery queries

I. Count the number of records with each different schema

In [7]:
for df in dfs:
    print(df.dtypes, len(df))

_path             object
_write_ts         object
ts                object
uid               object
id.orig_h         object
id.orig_p          int64
id.resp_h         object
id.resp_p          int64
proto             object
duration         float64
orig_bytes       float64
resp_bytes       float64
conn_state        object
missed_bytes       int64
history           object
orig_pkts          int64
orig_ip_bytes      int64
resp_pkts          int64
resp_ip_bytes      int64
service           object
dtype: object 1021952
_path                 object
_write_ts             object
ts                    object
uid                   object
id.orig_h             object
id.orig_p              int64
id.resp_h             object
id.resp_p              int64
trans_depth            int64
method                object
host                  object
uri                   object
version              float64
user_agent            object
request_body_len       int64
response_body_len      int64
status_code   